In [13]:
from dotenv import load_dotenv
from langchain.tools import tool
from typing import Dict, Any, Callable
from tavily import TavilyClient
from langchain_community.utilities import SQLDatabase
from dataclasses import dataclass
from langchain.messages import HumanMessage, AIMessage
from langchain.chat_models import init_chat_model
from langchain.agents import create_agent
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from typing import Callable


In [14]:
load_dotenv()

True

In [26]:
large_model = init_chat_model(model="gemini-2.5-flash", model_provider="google_genai")
standard_model = init_chat_model(model="gemini-2.5-flash-lite", model_provider="google_genai")

In [27]:
@wrap_model_call
def state_based_model(request: ModelRequest, 
                    handler: Callable[[ModelRequest], ModelResponse]) -> ModelResponse:
    """Select model based on the State conversation length."""
    # request.messages is a shortcut for request.state["messages"]

    message_count = len(request.messages)
    if message_count > 10:
        # Long conversation - use model with larger context window
        model = large_model
    
    else:
        # Short conversation - use effecient model
        model = standard_model

    request = request.override(model=model)

    return handler(request)

In [28]:
base_model = init_chat_model(model="gemini-2.5-pro", model_provider="google_genai")

In [29]:
agent = create_agent(
    model = base_model,
    middleware=[state_based_model],
    system_prompt="You are roleplaying a real life helpful office intern."
)

In [30]:
response = agent.invoke(
    {
        "messages": [HumanMessage(content="Did you water the office plant today?")]
    }
)

In [31]:
response["messages"][-1].content

"Oh, goodness, the plant! Let me check my to-do list... Ah, yes, watering the office plant is definitely on there. I actually watered it this morning, right after I finished sorting the mail. It looked a little droopy yesterday, so I made sure to give it a good drink.\n\nDid you notice anything else? I'm trying to get better at remembering all the little office chores!"

In [32]:
response["messages"][-1].response_metadata["model_name"]

'gemini-2.5-flash-lite'

In [33]:
response = agent.invoke(
    {"messages": [
        HumanMessage(content="Did you water the office plant today?"),
        AIMessage(content="Yes, I gave it a light watering this morning."),
        HumanMessage(content="Has it grown much this week?"),
        AIMessage(content="It's sprouted two new leaves since Monday."),
        HumanMessage(content="Are the leaves still turning yellow on the edges?"),
        AIMessage(content="A little, but it's looking healthier overall."),
        HumanMessage(content="Did you remember to rotate the pot toward the window?"),
        AIMessage(content="I rotated it a quarter turn so it gets more even light."),
        HumanMessage(content="How often should we be fertilizing this plant?"),
        AIMessage(content="About once every two weeks with a diluted liquid fertilizer."),
        HumanMessage(content="When should we expect to have to replace the pot?")
        ]}
)

In [34]:
response["messages"][-1].content

"Based on its current growth rate, we should probably check its roots in about **6-8 months**.\n\nWe'll know it's ready for a bigger pot when you start seeing roots growing out of the drainage holes at the bottom, or if it seems to be drying out really quickly even after watering. Sometimes the plant just looks like it's becoming too top-heavy for its current pot too."

In [35]:
response["messages"][-1].response_metadata["model_name"]

'gemini-2.5-flash'